In [1]:
import pandas as pd
import os 
import h3 as h3
import matplotlib.pyplot as plt
import plotly
import folium
import webbrowser
from folium import Map

In [2]:
from IPython.core.display import display, HTML
import json
from pandas.io.json import json_normalize
import numpy as np

import statistics

from geojson.feature import *
# from area import area

import copy
from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster
import branca.colormap as cm
from branca.colormap import linear

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image, display
from IPython.utils.text import columnize
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
# parl_const = pd.read_csv('~/Downloads/Book3.csv')    # parliamentary constituencies
# df = pd.read_csv("~/Downloads/9999_pc_with_latlongs.csv")   # Food bank locations
df = pd.read_csv('~/Downloads/Book3.csv')    # parliamentary constituencies
ori = pd.read_csv('~/Downloads/Book3.csv')    # parliamentary constituencies

print(df.head())

         constituency     lat    lng  population  random
0            Aberavon  51.600 -3.812       68649    3653
1           Aberconwy  53.302 -3.805       59207    3481
2      Aberdeen North  57.176 -2.135      101461     758
3      Aberdeen South  57.095 -2.133       94831    1950
4  Airdrie and Shotts  55.845 -3.892       86736    2599


In [4]:
# Simplify column names
# parl_const.columns = ['lat','lng']
# test = parl_const.sample(frac=0.05, replace=False, random_state=1)
# test.plot(x='lng',y='lat',style='.',alpha=0.2,figsize=(12,12));
# plt.title('UK constituencies');

In [5]:


def counts_by_hexagon(df, resolution):

    '''Use h3.geo_to_h3 to index each data point into the spatial index of the specified resolution.
      Use h3.h3_to_geo_boundary to obtain the geometries of these hexagons'''

    df = df[["lat","lng"]]

    df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["lat"], row["lng"], resolution), axis = 1)

    df_aggreg = df
    df_aggreg['population'] = ori['population']
    df_aggreg['constituency'] = ori['constituency']

    df_aggreg.columns = ["lat", "lng", "hex_id", "population", "constituency"]

    df_aggreg["geometry"] =  df_aggreg.hex_id.apply(lambda x: 
                                                           {    "type" : "Polygon",
                                                                 "coordinates": 
                                                                [h3.h3_to_geo_boundary(x,geo_json=True)]
                                                            }
                                                        )

    return df_aggreg


def hexagons_dataframe_to_geojson(df_hex, file_output = None):
    """
    Produce the GeoJSON for a dataframe that has a geometry column in geojson 
    format already, along with the columns hex_id and value

    Ex counts_by_hexagon(data)
    """    
    list_features = []

    for i,row in df_hex.iterrows():
        feature = Feature(geometry = row["geometry"] , id=row["hex_id"], properties = {"population" : row["population"]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    geojson_result = json.dumps(feat_collection)

    #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)

    return geojson_result

def choropleth_map(df_aggreg, border_color = 'black', fill_opacity = 0.7, initial_map = None, with_legend = False,
               kind = "linear"):

    """
    Creates choropleth maps given the aggregated data.
    """    
    #colormap
    min_value = df_aggreg["population"].min()
    max_value = df_aggreg["population"].max()
    m = round ((min_value + max_value ) / 2 , 0)

    #take resolution from the first row
    res = h3.h3_get_resolution(df_aggreg.loc[0,'hex_id'])

    if initial_map is None:
        initial_map = Map(location= [51.53, 0.11], zoom_start=7, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )


    #the colormap 
    #color names accepted https://github.com/python-visualization/branca/blob/master/branca/_cnames.json
    if kind == "linear":
        custom_cm = cm.LinearColormap(['green','yellow','red'], vmin=min_value, vmax=max_value)
    elif kind == "outlier":
        #for outliers, values would be -11,0,1
        custom_cm = cm.LinearColormap(['blue','white','red'], vmin=min_value, vmax=max_value)
    elif kind == "filled_nulls":
        custom_cm = cm.LinearColormap(['sienna','green','yellow','red'], 
                                      index=[0,min_value,m,max_value],vmin=min_value,vmax=max_value)


    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_hex = df_aggreg)

    #plot on map
    name_layer = "Choropleth " + str(res)
    if kind != "linear":
        name_layer = name_layer + kind

    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['population']),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)
    #add legend (not recommended if multiple layers)
    if with_legend == True:
        custom_cm.add_to(initial_map)



    return initial_map

def plot_scatter(df, metric_col, x='lng', y='lat', marker='.', alpha=1, figsize=(16,12), colormap='viridis'):
    """
    Scatter plot function for h3 indexed objects
    """    
    df.plot.scatter(x=x, y=y, c=metric_col, title=metric_col
                    , edgecolors='none', colormap=colormap, marker=marker, alpha=alpha, figsize=figsize);
    plt.xticks([], []); plt.yticks([], [])


In [6]:
# Counts how many points are within the hex
myres = 6
df_aggreg = counts_by_hexagon(df = df, resolution = myres)      #Use 4 to tile the whole of uk in an aesthetically pleasing way
df_aggreg.sort_values(by = "population", ascending = False, inplace = True)

# Creates a map using Folium
hexmap = choropleth_map(df_aggreg = df_aggreg, with_legend = True)
hexmap.save('downloads')
hexmap

In [7]:
df_aggreg

,lat,lng,hex_id,population,constituency,geometry
616,51.53034,0.01352,86194e69fffffff,188275,West Ham,"{'type': 'Polygon', 'coordinates': [((-0.01369..."
450,51.50718,-0.03515,86194ad37ffffff,167368,Poplar and Limehouse,"{'type': 'Polygon', 'coordinates': [((-0.06929..."
192,51.53002,0.05504,86194e68fffffff,163730,East Ham,"{'type': 'Polygon', 'coordinates': [((0.071597..."
352,53.48600,-2.20903,8619424dfffffff,162656,Manchester Central,"{'type': 'Polygon', 'coordinates': [((-2.19869..."
285,51.54281,-0.13937,86195da4fffffff,159344,Holborn and St Pancras,"{'type': 'Polygon', 'coordinates': [((-0.15469..."
...,...,...,...,...,...,...
184,52.81200,-4.12050,86195e527ffffff,61421,Dwyfor Meirionnydd,"{'type': 'Polygon', 'coordinates': [((-4.14432..."
104,58.27700,-3.77900,86192bd77ffffff,60554,"Caithness, Sutherland and Easter Ross","{'type': 'Polygon', 'coordinates': [((-3.78909..."
1,53.30200,-3.80500,861953b6fffffff,59207,Aberconwy,"{'type': 'Polygon', 'coordinates': [((-3.80237..."
435,59.76688,-2.61588,86192c1a7ffffff,45180,Orkney and Shetland,"{'type': 'Polygon', 'coordinates': [((-2.63210..."


In [8]:
# h3.h3_to_geo('85194ad3fffffff')

In [9]:
def show_map(themap, file_name):
    themap.save(file_name)
    wb = webbrowser.open('file://' + os.path.realpath(file_name), new=2)     #opens in new tab. using a new of 1 opens in a new window

In [10]:
show_map(hexmap, "h3_level:_{}_.html".format(myres))

In [11]:
column2_as_series = set(df_aggreg['hex_id'])
column2_as_series

{'861821937ffffff',
 '8618219b7ffffff',
 '861825067ffffff',
 '8618250a7ffffff',
 '861825237ffffff',
 '86182d2efffffff',
 '861870cb7ffffff',
 '861874027ffffff',
 '86187404fffffff',
 '86187460fffffff',
 '8618746d7ffffff',
 '861874757ffffff',
 '861874857ffffff',
 '861874a5fffffff',
 '861874ac7ffffff',
 '861874acfffffff',
 '86190905fffffff',
 '86190929fffffff',
 '8619092efffffff',
 '86190931fffffff',
 '861909327ffffff',
 '861909347ffffff',
 '861909717ffffff',
 '861909847ffffff',
 '861909a1fffffff',
 '861909a2fffffff',
 '861909a57ffffff',
 '861909a5fffffff',
 '861909a97ffffff',
 '861909ac7ffffff',
 '861909acfffffff',
 '861909adfffffff',
 '861909ae7ffffff',
 '861909af7ffffff',
 '86190c2c7ffffff',
 '86190d15fffffff',
 '86190d89fffffff',
 '86190e8a7ffffff',
 '86192b26fffffff',
 '86192bd77ffffff',
 '86192c1a7ffffff',
 '861940407ffffff',
 '861940427ffffff',
 '86194042fffffff',
 '86194044fffffff',
 '8619404b7ffffff',
 '861940567ffffff',
 '861940667ffffff',
 '861940697ffffff',
 '861940737ffffff',


In [12]:
len(column2_as_series)

599

In [13]:
# Work out mean number of people in each hexagon

df_aggreg["population"].mean()

102227.04

In [14]:
# Calculate the standard deviation for the population

df_aggreg["population"].std()

16391.88006154793